In [87]:
from datasets import load_from_disk
import os
import random

In [88]:
path = "/home/raj/Lab/Dataset"
dataset = os.listdir(path)
dataset.sort()
dataset

['bh',
 'bn_dev',
 'bn_train_5splits_split1',
 'bn_train_5splits_split2',
 'bn_train_5splits_split3',
 'bn_train_5splits_split4',
 'bn_train_5splits_split5']

In [89]:
utt2lang_path = "RESPIN_ASRU_Challenge_2023/corpus/bh/dev/utt2lang"
utt2lang = {}
with open(utt2lang_path) as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        first_space = line.find("\t")
        audio_id = line[:first_space]
        lang = line[first_space+1:]
        utt2lang[audio_id] = lang
utt2lang          

{'281474976713362_387672_281474977649608': 'D3',
 '281474976713362_388437_281474977649613': 'D3',
 '281474976713362_389124_281474977649633': 'D3',
 '281474976713362_388598_281474977667011': 'D3',
 '281474976713362_389827_281474977667017': 'D3',
 '281474976713362_387705_281474977690949': 'D3',
 '281474976713362_389545_281474977690960': 'D3',
 '281474976713362_389349_281474977690974': 'D3',
 '281474976713362_389772_281474977692199': 'D3',
 '281474976713362_389516_281474977692214': 'D3',
 '281474976713362_388686_281474977692736': 'D3',
 '281474976713362_388900_281474977692749': 'D3',
 '281474976713362_389477_281474977692773': 'D3',
 '281474976713362_389410_281474977693518': 'D3',
 '281474976713209_374602_281474977707171': 'D2',
 '281474976713209_375988_281474977707173': 'D2',
 '281474976713209_375323_281474977707174': 'D2',
 '281474976713209_375144_281474977707176': 'D2',
 '281474976713209_377146_281474977707177': 'D2',
 '281474976713209_376820_281474977707179': 'D2',
 '281474976713209_37

In [90]:
utt2dur_path = "RESPIN_ASRU_Challenge_2023/corpus/bh/dev/utt2dur"
utt2dur = {}
with open(utt2dur_path) as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        first_space = line.find(" ")
        audio_id = line[:first_space]
        dur = line[first_space+1:]
        utt2dur[audio_id] = dur
utt2dur          

{'16777256_366501_281474982287542': '9.05',
 '16777256_366757_281474982287518': '8.01',
 '16777256_367246_281474982287523': '9.27',
 '16777256_367340_281474982287529': '8.15',
 '16777256_368351_281474982287539': '6.29',
 '16777256_370161_281474982287512': '4.45',
 '16777256_370722_281474982287530': '8.49',
 '16777256_370773_281474982287543': '8.89',
 '16777256_370911_281474982287522': '7.93',
 '16777256_371723_281474982287540': '5.11',
 '16777256_372165_281474982287537': '4.57',
 '16777256_372194_281474982287510': '4.39',
 '16777387_380212_281474982217803': '2.34',
 '16777387_380232_281474982217782': '2.46',
 '16777387_380258_281474982218116': '3.26',
 '16777387_380280_281474982218588': '3.29',
 '16777387_380355_281474982218100': '3.01',
 '16777387_380367_281474982218585': '2.48',
 '16777387_380382_281474982217767': '2.76',
 '16777387_380443_281474982217800': '2.06',
 '16777387_380466_281474982217780': '2.32',
 '16777387_380488_281474982217770': '2.05',
 '16777387_380511_28147498221780

In [91]:
def read_text_file(path):
    data = []
    dict= {}
    with open(path,'r') as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()  # Remove leading/trailing whitespace and newline characters
            first_space = line.find(" ")
            audio_info = line[:first_space].split('_')
            label = line[first_space+1:]
            spkids = audio_info[0]
            text_id = audio_info[1]
            recording_id = audio_info[2] 
            lang_id = utt2lang[line[:first_space]]
            dict[recording_id] = [spkids,text_id,recording_id,label,lang_id]
            data.append([spkids,text_id,recording_id,label,lang_id])
    return data,dict

In [92]:
dev_text_Path = "/home/raj/MADASR-Competition/RESPIN_ASRU_Challenge_2023/corpus/bh/dev/text"
dev_data,dev_dict = read_text_file(dev_text_Path)

In [93]:
dev_data[120]

['16777398',
 '380748',
 '281474982295585',
 'हमार खाता में मोबाइल नम्बर जुड़ल हो',
 'D2']

In [94]:
len(dev_data)

1262

In [95]:
selected_dev_dataset = []
dataset_path = f"/home/raj/Lab/Dataset/bh/bh_dev/dev/"
dev_files = os.listdir(dataset_path)
len(dev_files)
print(len(dev_files))
for name in dev_files:
    audio_info = dev_dict[name[:-4]]
    if(audio_info[4]=='D3' and float(utt2dur[f"{audio_info[0]}_{audio_info[1]}_{audio_info[2]}"])<30):
        audio_info[2] = os.path.join(dataset_path,name) #set path instead of recording id now...
        selected_dev_dataset.append(audio_info)

1262


In [96]:
selected_dev_dataset

[]

In [97]:
len(selected_dev_dataset)

0

In [98]:
from datasets import Dataset, DatasetDict

def convert_list_to_dataset_HF(dataset):
    data_dict = {
    "spkids": [item[0] for item in dataset],
    "text_id": [item[1] for item in dataset],
    "path" : [item[2] for item in dataset],
    "label" : [item[3] for item in dataset],
    "lang_id" : [item[4] for item in dataset]
    }
    return Dataset.from_dict(data_dict)
dev_data_hf = convert_list_to_dataset_HF(selected_dev_dataset)

In [99]:
dev_data_hf

Dataset({
    features: ['spkids', 'text_id', 'path', 'label', 'lang_id'],
    num_rows: 0
})

In [100]:

# dev_data_hf[200]

IndexError: Invalid key: 200 is out of bounds for size 0

In [ ]:
len(dev_data_hf)

471

In [ ]:
from datasets import Audio
dev_dataset = dev_data_hf.cast_column("path", Audio(sampling_rate=16000))

In [ ]:
dev_dataset

Dataset({
    features: ['spkids', 'text_id', 'path', 'label', 'lang_id'],
    num_rows: 471
})

In [ ]:
dev_dataset[200]

{'spkids': '16778697',
 'text_id': '388332',
 'path': {'path': '/home/raj/Lab/Dataset/bh/bh_dev/dev/281474982805100.wav',
  'array': array([0.00067139, 0.00067139, 0.00271606, ..., 0.0637207 , 0.0241394 ,
         0.00991821]),
  'sampling_rate': 16000},
 'label': 'पैसे तुरंत भेजे खातिर कौन तरीका बा',
 'lang_id': 'D3'}

In [ ]:
def prepare_dataset(batch):
    audio = batch["path"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["label"]).input_ids
        batch["lang_id"] = batch["lang_id"] 
    return batch

In [ ]:
from transformers import Wav2Vec2Processor
processor = Wav2Vec2Processor.from_pretrained("wav2vec2-again-3,4,5-fine-tune-bhojpuri")
dev_dataset = dev_dataset.map(prepare_dataset,remove_columns=['spkids','text_id','path','label'] ,num_proc=4)

Map (num_proc=4):   0%|          | 0/471 [00:00<?, ? examples/s]

/home/raj/anaconda3/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:154: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/raj/anaconda3/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:154: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/raj/anaconda3/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:154: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `t

In [ ]:
dev_dataset

Dataset({
    features: ['lang_id', 'input_values', 'input_length', 'labels'],
    num_rows: 471
})

In [ ]:
dev_dataset[0]

{'lang_id': 'D3',
 'input_values': [0.10473380237817764,
  0.07853224128484726,
  0.04379533603787422,
  0.04002390056848526,
  0.04300135001540184,
  0.06761492788791656,
  0.06027055159211159,
  0.04657428711652756,
  0.021166717633605003,
  0.01580730825662613,
  0.021563712507486343,
  0.023747174069285393,
  0.008264436386525631,
  -0.010394248180091381,
  -0.003843858605250716,
  0.0036990137305110693,
  0.018189268186688423,
  0.01838776469230652,
  0.0064779664389789104,
  -0.014364180155098438,
  -0.018929602578282356,
  -0.0006679124780930579,
  0.022754691541194916,
  0.03287801891565323,
  0.009653912857174873,
  -0.007019805256277323,
  -0.031831882894039154,
  -0.032228875905275345,
  -0.03997024521231651,
  -0.03659580275416374,
  -0.03679430112242699,
  0.0013170539168640971,
  0.04617729410529137,
  0.0787307396531105,
  0.08150969445705414,
  0.07972322404384613,
  0.07674577087163925,
  0.0896480530500412,
  0.1277594119310379,
  0.14979253709316254,
  0.148204565048

In [ ]:
dev_dataset.save_to_disk("D4_dev_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/471 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_from_disk

data = load_from_disk("D4_dev_dataset")
data

Dataset({
    features: ['lang_id', 'input_values', 'input_length', 'labels'],
    num_rows: 471
})

In [ ]:
data[0]

{'lang_id': 'D3',
 'input_values': [0.10473380237817764,
  0.07853224128484726,
  0.04379533603787422,
  0.04002390056848526,
  0.04300135001540184,
  0.06761492788791656,
  0.06027055159211159,
  0.04657428711652756,
  0.021166717633605003,
  0.01580730825662613,
  0.021563712507486343,
  0.023747174069285393,
  0.008264436386525631,
  -0.010394248180091381,
  -0.003843858605250716,
  0.0036990137305110693,
  0.018189268186688423,
  0.01838776469230652,
  0.0064779664389789104,
  -0.014364180155098438,
  -0.018929602578282356,
  -0.0006679124780930579,
  0.022754691541194916,
  0.03287801891565323,
  0.009653912857174873,
  -0.007019805256277323,
  -0.031831882894039154,
  -0.032228875905275345,
  -0.03997024521231651,
  -0.03659580275416374,
  -0.03679430112242699,
  0.0013170539168640971,
  0.04617729410529137,
  0.0787307396531105,
  0.08150969445705414,
  0.07972322404384613,
  0.07674577087163925,
  0.0896480530500412,
  0.1277594119310379,
  0.14979253709316254,
  0.148204565048

In [3]:
! kenlm/build/bin/query "d1_6_gram_correct_additional_train.arpa" < "Analysis_dataset/d1_dev_text.txt"

Loading the LM will be faster if you build a binary file.
Reading d1_6_gram_correct_additional_train.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
अब=1413 2 -2.5848434	इ=182 3 -1.4248621	सब=120 4 -1.1254691	मशीन=829 5 -0.6519938	के=11 6 -0.6304921	आगइला=0 1 -6.0875235	से=26 1 -1.8567044	कटाई=559 2 -3.2227407	पिटाई=1421 3 -1.5111886	घंटा=622 4 -0.79207605	भर=79 5 -0.58079016	में=7 6 -0.30669382	हो=92 6 -0.9807301	जात=149 6 -0.8673192	हवे=35 4 -0.4157685	</s>=2 5 -0.06929276	Total: -23.108486 OOV: 1
मछरी=140 2 -2.6556551	बाजार=908 3 -1.0218719	में=7 4 -0.01664045	सब=120 5 -2.3779576	लोग=137 6 -0.81470895	ना=272 6 -1.0351164	जा=591 6 -0.74078244	पाई=3636 6 -0.7999668	</s>=2 6 -0.3883178	Total: -9.851018 OOV: 0
माटी=522 2 -2.3255098	के=11 3 -0.58339924	पैदावार=2099 3 -3.722763	कईसे=1591 1 -3.744032	बढ़ावल=4061 2 -2.0797734	जाला=81 3